## 6. Target
We compared different models. The Random Forest Regressor had the best results, so this is the model we will use 
to make the predictions for the target. <br>
The predictions will be saved in the file ```RandomForest_Predictions.csv```. This file is needed for the visualizations 
in our dashboard.<br>
We make the predictions with the models that we saved on the disk as ```.pkl``` files. <br>
Alternatively we coud load the models from MLflow:<br>
```python
     model = mlflow.pyfunc.load_model(model_path)
     model.predict(model_input)
```    


In [18]:
## load modules
import pandas as pd
import sys
sys.path.append("..")
from modeling.features import get_feature_combinations
from modeling.functions import predict_func, train_test_split_features, scaler_func
from sklearn.preprocessing import MinMaxScaler
import pickle

In [19]:
## read data
data = pd.read_csv('../data/GEFCom2014Data/Wind/clean_data.csv', 
                    parse_dates= ['TIMESTAMP'],
                    index_col= 'TIMESTAMP' )

In [20]:
# train-test-split 
data_train = data[:'2013-07-01 00:00:00']
data_test = data['2013-07-01 01:00:00':]

In [21]:
# load model parameter
df_model = pd.read_csv(f'../results/RandomForestRegressor.csv'mar

SyntaxError: invalid syntax (4226515735.py, line 2)

In [ ]:
# make the predictions and save them in df_pred
df_pred = pd.DataFrame(index=data_test[data_test.ZONEID == 1].index)

for zone in data.ZONEID.unique():
    # obtain features from the current feature combination
    fc = df_model[df_model.ZONE == 'ZONE{}'.format(str(zone))]['FC'].values[0]

    # obtain data frame for zone
    data_train_zone = data_train[data_train.ZONEID == zone]
    data_test_zone = data_test[data_test.ZONEID == zone]

    # split in X and y
    features = get_feature_combinations()[fc]
    X_train, X_test, y_train, y_test = train_test_split_features(data_train, data_test, zone, features)  
    X_train, X_test= scaler_func(X_train, X_test, MinMaxScaler())

    # load the model for current windfarm 
    model = pickle.load(open('../models/model{}.pkl'.format(zone), 'rb'))
   
    # make prediction
    y_pred = predict_func(model, X_test, y_test)
    df_pred[f'Zone {zone}'] = y_pred
    

In [ ]:
df_pred

In [ ]:
# save the predictions in ```RandomForest_Predictions.csv ```
df_pred.to_csv('../results/RandomForest_Predictions.csv')